# Florida Keys Taxonomic Analysis
Taxonomic Analysis - Florida Keys

### importing modules

In [2]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import cartopy as crt
import geoplot as gplt
%matplotlib inline

In [2]:
import plotly.express as px
import plotly.graph_objects as go

We'll not need `pyobis` for this notebook since we have an attached csv data source that had already been fetched using it.

In [15]:
df = pd.read_csv("../input/obis-florida-keys-occurrence-records-19972012/florida-keys-1997-2012.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (49,77,79,95,97,99,100,104,105,112,113,115,117,134,149,150,158,159,160,165,166,167,168,169,170,172,173,174,175,177,178,179,181,182,183,184,185,186,188,189,190,191,192,193,194,196) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Notes before we proceed
> To be specific, these are notes for me to read before I proceed with the analysis. It helps me to stay focused in what work I'll do.

This is a **seascapes analysis** and not a **species analysis** in particular.

1. seascapes as a product, we need not to generate <- ERDDAP server
2. depth change due to surface phenomenon?
3. if spatial smapling did change? did the sampling change over time?
4. characterize each seascapes?
5. holvert index, refractoring curve. investigate richness. adjusting for sampling bias
6. seascapes -> surface parameters, may have meaning to depth
7. for things to be easier, focus on surface at the time
8. might add a biologist into the discussion
9. corals in area where certain seascapes are dominant <- related to surface phenomenon
10. ideal taxon, phytoplankton since attached to seascapes <- more or less productive based on seascape
11. one that moves and doesn't stay at the same position
12. investigate MoF records as well
13. may also look at GBIF
14. research questions
+ what species live in these different seascapes?
    + taxonomic dist, whether a part. seascapes suggest occurrence of one species
+ what are the biological traits of these species?
+ different in the amount of biodiversity they support?

15. as seascapes made by satellite people not biodiversity people; it is a common research question.
16. how to adjust sampling bias?
+ ES50 metric; https://esajournals.onlinelibrary.wiley.com/doi/epdf/10.2307/1934145?saml_referrer might be helpful

17. potential data source for seascape data: [ERDDAP](http://erddap.com/#search=seascape)
+ 8-day is best <- temporal resolution
+ 1 km resolution <- spatial resolution
+ resolutions should be comparable with OBIS data, 
    + check if that uncertainity is whithin a seascape analysis
18. what we can do in the research
+ might exclude if uncertainity is huge, need not skip blanks
+ perform a correlation test
+ what does individualDensity mean?
    + sometimes, NERC (https://vocab.nerc.ac.uk/search_nvs/);
    + a deep rabbithole to get deep down
+ individualCount v/s organismQuantity? 
    + biodata mobilisation (https://ioos.github.io/bio_mobilization_workshop/04-create-schema/index.html)
+ there could be different organismQuantity?
19. Reading Sources
+ Read here about OBIS: https://github.com/iobis/manual
+ MoF Viewer: https://mof.obis.org/, not well standardised
+ we can ask questions on slack <- https://github.com/ioos/bio_data_guide
    + very much active and experts stay in here

### Cleaning the data
Our data looks messy and before we prepare our sunburst, let's clean our dataset first. 

We'll remove/replace some NaNs and remove unwanted rows & columns.

In [19]:
df.drop("Unnamed: 0", axis=1, inplace=True)

In [20]:
df.head()

,infraphylum,country,date_year,habitat,references,scientificNameID,scientificName,dropped,gigaclassid,aphiaID,...,startDayOfYear,otherCatalogNumbers,footprintSRS,associatedSequences,locationRemarks,behavior,verbatimDepth,taxonConceptID,subtribeid,subtribe
0,Gnathostomata,United States,2009,Spur and Groove low Relief,doi:10.1016/j.fishres.2011.01.012,urn:lsid:marinespecies.org:taxname:276677,Stegastes partitus,False,10194.0,276677,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,2011,NaN,https://www.ncei.noaa.gov/erddap/tabledap/deep...,urn:lsid:marinespecies.org:taxname:164811,Demospongiae,False,NaN,164811,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,United States,2008,Deep Fore Reef,NaN,urn:lsid:marinespecies.org:taxname:210726,Millepora alcicornis,False,NaN,210726,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Gnathostomata,United States,2010,Spur and Groove low Relief,doi:10.1016/j.fishres.2011.01.012,urn:lsid:marinespecies.org:taxname:275764,Halichoeres garnoti,False,10194.0,275764,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Gnathostomata,United States,2012,Spur and Groove High Relief,doi:10.1016/j.fishres.2011.01.012,urn:lsid:marinespecies.org:taxname:276025,Pomacanthus paru,False,10194.0,276025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
df["species"] = df["species"].fillna("other")

> **The heirarchy we are following** 
> 
> kingdom -> phylum -> order -> class -> genus -> species

In [ ]:
# phylum nan
df[]
# order nan
# class nan
# genus nan
# species nan

In [34]:
# when kingdom is NaN
df[df["kingdom"].isna()].T

,774841
infraphylum,NaN
country,United States
date_year,2006
habitat,NaN
references,NaN
...,...
behavior,NaN
verbatimDepth,NaN
taxonConceptID,NaN
subtribeid,NaN


Let us drop this.

In [ ]:
df.loc[774841:774841,:]

## Taxonomic Distribution
Using plotly sunburst.

In [26]:
fig = px.sunburst(df,
                  path=["kingdom", "phylum", "genus","species"],
                  width=750, height=750,
                  title="Taxonomic Pie Chart",
                  )
fig.show()

ValueError: ('None entries cannot have not-None children', kingdom    Animalia
phylum     Annelida
genus           nan
species       other
Name: 104, dtype: object)